In [1]:
import mysql.connector as connector

In [7]:
from mysql.connector import pooling, Error

In [ ]:
try:
    # Connection configuration
    connection_config_dict = {
        'host': 'localhost',
        'user': 'root',
        'database': 'little_lemon_db',
        'autocommit': True
    }

    # Connect to the database using mysql.connector
    connection = connector.connect(**connection_config_dict)

    # Create a connection pool
    pool = pooling.MySQLConnectionPool(pool_name="little_lemon_pool",
                                       pool_size=2,
                                       **connection_config_dict)

    print("The connection pool is created with the name:", pool.pool_name, "and the pool size is:", pool.pool_size)

    # Get a cursor object from the connection
    cursor = connection.cursor()

    if connection.is_connected():
        
        # Use the little_lemon_db database
        use_database_query = """USE little_lemon_db;"""
        cursor.execute(use_database_query)
        print("Using little lemon database")

        # Define and execute the BasicSalesReport procedure
        BasicSalesReport_procedure = """
        CREATE PROCEDURE BasicSalesReport()
        BEGIN
            SELECT SUM(BillAmount) AS `Total sales`, AVG(BillAmount) AS `Average sale`, 
                   MIN(BillAmount) AS `Minimum bill paid`, MAX(BillAmount) AS `Maximum bill paid` 
            FROM Orders;
        END
        """
        cursor.execute(BasicSalesReport_procedure)

        # Call the BasicSalesReport procedure
        cursor.callproc("BasicSalesReport")

        # Fetch and print results
        for result in cursor.stored_results():
            print(result.fetchall())

           
except Error as e:
    # Handle generic MySQL errors
    print("MySQL Error:", e)

except Exception as ex:
    # Handle any other type of exception not caught by previous blocks
    print("Unknown Error:", ex)

finally:
    if 'cursor' in locals() and cursor:
        cursor.close()
    if 'connection' in locals() and connection.is_connected():
        connection.close()
        print("MySQL connection is closed")
